In [ ]:
%pip install pandas requests

In [ ]:
import pandas as pd
import requests
import os 
import time

# Read in file 
df = pd.read_csv('books.csv')

# Inspect df
df.head()
df.shape

(18044, 3)

In [3]:
# Rename columns
df = df.rename(columns={"book_title": "title", "book_genres": "genres"})

In [5]:
# Drop rows with missings on title or author
df = df.dropna(subset=["title", "author"])

In [ ]:
# Select 10 random rows to test if PPLX code works
df10 = df.sample(n=10, random_state=42)
df10.head()

# Drop genres
df10 = df10.drop(columns="genres")

In [ ]:
system_prompt = (
    "You are a helpful literary assistant. Your job is to search the internet and assess whether a book features a female main character. "
    "The character does not need to be the sole protagonist but must play a central role in the plot, "
    "with more significance than a secondary or supporting character."
)

user_prompt = "Does the book {title} by {author} have female main character?"

In [ ]:
# Setting my API key
YOUR_API_KEY = os.environ["PERPLEXITY_API_KEY"]

# Saying hi it's me, and this is what I'll be sending
headers = {
    "Authorization": f"Bearer {YOUR_API_KEY}",
    "Content-Type": "application/json"
}

# This is where I want to send it to
url = "https://api.perplexity.ai/chat/completions"

# I'll store the responses I get in this (as of now, empty) list
response = []

# # Setting up what I want to send to Perplexity
# payload = {
#     "model": "sonar",
#     "messages": [
#         {
#             "role": "system",
#             "content": system_prompt
#         },
#         {
#             "role": "user",
#             "content": user_prompt
#         }
#     ],
#     "max_tokens": 123,  # Limits the length of the response
#     "temperature": 0.2,  # Controls randomness in the response
#     "top_p": 0.9,
#     "search_domain_filter": ["<any>"],
#     "return_images": False,
#     "return_related_questions": False,
#     "search_recency_filter": "<string>",
#     "top_k": 0,
#     "stream": False,
#     "presence_penalty": 0,
#     "frequency_penalty": 1,
#     "response_format": {},
#     "web_search_options": {"search_context_size": "high"}
# }




# # Send the request to the Perplexity API
# response = requests.post(url, json=payload, headers=headers)

# # Printing out the response and error handling 
# if response.status_code == 200:
#     response_json = response.json()
#     print(response_json)
# else:
#     print(f"Error: {response.status_code}")

In [ ]:
# Set your API key
YOUR_API_KEY = os.environ["PERPLEXITY_API_KEY"]

# Define your system prompt
system_prompt = (
    "You are a helpful literary assistant. Your job is to search the internet and assess whether a book features a female main character. "
    "The character does not need to be the sole protagonist but must play a central role in the plot, "
    "with more significance than a secondary or supporting character."
)

# Define your user prompt
user_prompt_template = "Does this book have a female main character? Title: {title}, Author: {author}"

# Set up headers
headers = {
    "Authorization": f"Bearer {YOUR_API_KEY}",
    "Content-Type": "application/json"
}

# Your URL for the request
url = "https://api.perplexity.ai/chat/completions"

# Create an empty list to store the answers
responses = []

# Loop through each row in your dataframe df10
for index, row in df10.iterrows():
    # Extract title and author from each row
    title = row['title']
    author = row['author']
    
    # Format the user prompt with the current title and author
    user_prompt = user_prompt_template.format(title=title, author=author)
    
    # Prepare the payload for the API request
    payload = {
        "model": "sonar",
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        "max_tokens": 123,  # Limits the length of the response
        "temperature": 0.2,  # Controls randomness in the response
        "top_p": 0.9,
        "search_domain_filter": ["<any>"],
        "return_images": False,
        "return_related_questions": False,
        "search_recency_filter": "<string>",
        "top_k": 0,
        "stream": False,
        "presence_penalty": 0,
        "frequency_penalty": 1,
        "response_format": {},
        "web_search_options": {"search_context_size": "high"}
    }
    
    # Send the request to the Perplexity API
    response = requests.post(url, json=payload, headers=headers)
    
    # Check the response
    if response.status_code == 200:
        response_json = response.json()
        # Extract the answer from the response (this might vary depending on the response format)
        answer = response_json.get('choices', [{}])[0].get('message', {}).get('content', 'No answer found')
        responses.append(answer)  # Store the answer
    else:
        # Handle errors by appending an error message
        responses.append(f"Error: {response.status_code}")

# Add the responses to your original dataframe
df10['female_main_character'] = responses

# Print the updated dataframe to check the results
print(df10)
